# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [3]:
PREDICTION_METHOD="Module-based"

In [4]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [5]:
OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
display(OUTPUT_DATA_DIR)
OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data')

In [6]:
OUTPUT_RAW_DATA_DIR = Path(OUTPUT_DATA_DIR, "raw")
display(OUTPUT_RAW_DATA_DIR)
OUTPUT_RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw')

In [7]:
OUTPUT_PROJ_DATA_DIR = Path(OUTPUT_DATA_DIR, "proj")
display(OUTPUT_PROJ_DATA_DIR)
OUTPUT_PROJ_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj')

In [8]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [9]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [10]:
gold_standard.shape

(998, 3)

In [11]:
gold_standard.head()

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [12]:
doids_in_gold_standard = set(gold_standard['trait'])

# Load LINCS

## Projected data

In [13]:
# TODO: hardcoded
input_file = Path(
    OUTPUT_PROJ_DATA_DIR,
    "lincs-projection.pkl"
).resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj/lincs-projection.pkl')

In [14]:
lincs_projection = pd.read_pickle(input_file)

In [15]:
lincs_projection.shape

(987, 1170)

In [16]:
lincs_projection.head()

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
LV1,0.036115,-0.358610,0.091067,0.008923,0.046469,0.136747,0.036151,0.034024,-0.027445,0.060644,...,0.069930,0.021537,0.020511,-0.006250,-0.002660,-0.017210,-0.004029,-0.145490,0.102504,0.057621
LV2,0.012281,0.009738,0.004650,-0.006554,0.021758,0.010425,0.000794,-0.000603,0.011534,-0.004425,...,0.000344,0.001123,-0.012675,-0.004849,0.000375,0.002623,-0.003252,-0.066209,0.025378,0.012453
LV3,-0.005533,-0.059174,0.013454,0.008906,-0.004939,0.033663,-0.010045,0.003824,0.009814,-0.007492,...,-0.021941,-0.002417,0.000155,0.002941,-0.015100,-0.016372,0.012249,0.009216,0.018254,-0.003516
LV4,-0.004151,-0.032884,0.005934,-0.003991,-0.028524,0.002709,0.007001,0.024704,0.005058,-0.004345,...,0.013806,0.003197,0.013099,0.002643,-0.008836,0.023852,0.028957,0.014681,0.000826,-0.009739
LV5,-0.015156,-0.005276,0.025747,0.000346,-0.013932,-0.010911,0.041403,-0.018285,0.003507,-0.007621,...,-0.011577,-0.015956,-0.027614,-0.001707,-0.000542,-0.016918,0.001959,0.024147,0.013004,-0.014166


# Load S-PrediXcan

In [17]:
from entity import Trait

In [18]:
phenomexcan_input_file_list = [
    f for f in OUTPUT_PROJ_DATA_DIR.glob("*.pkl") if f.name.startswith(('smultixcan-', 'spredixcan-'))
]

In [19]:
display(len(phenomexcan_input_file_list))

51

# Predict drug-disease associations

In [20]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)
    
    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)
    print(f"  shape: {phenomexcan_projection.shape}")
    
    # prediction
    print(f"  predicting...")
    drug_disease_assocs = lincs_projection.T.dot(phenomexcan_projection)
    print(f"    shape: {drug_disease_assocs.shape}")
    drug_disease_assocs = Trait.map_to_doid(drug_disease_assocs, doids_in_gold_standard, combine="max")
    print(f"    shape (after DOID map): {drug_disease_assocs.shape}")
    assert drug_disease_assocs.index.is_unique
    assert drug_disease_assocs.columns.is_unique
    
    # build classifier data
    print(f"  building classifier data...")
    classifier_data = drug_disease_assocs\
        .unstack().reset_index()\
        .rename(columns={'level_0': 'trait', 'perturbagen': 'drug', 0: 'score'})
    assert classifier_data.shape == classifier_data.dropna().shape
    print(f"    shape: {classifier_data.shape}")
    display(classifier_data.describe())
    
    # save
    output_file = Path(
        OUTPUT_PREDICTIONS_DIR,
        f"{phenomexcan_input_file.stem}-prediction_scores.h5"
    ).resolve()
    print(f"    saving to: {str(output_file)}")
    
    classifier_data.to_hdf(output_file, mode="w", complevel=4, key='prediction')
    
    pd.Series({
        'method': PREDICTION_METHOD,
        'data': phenomexcan_input_file.stem,
    })\
    .to_hdf(output_file, mode="r+", key='metadata')
    
    print("")

spredixcan-mashr-zscores-Brain_Caudate_basal_ganglia-projection.pkl
  shape: (987, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


,score
count,411840.000000
mean,0.003594
std,0.023792
min,-1.005151
25%,-0.004901
50%,0.002989
75%,0.010917
max,1.190220


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-mashr-zscores-Brain_Caudate_basal_ganglia-projection-prediction_scores.h5

spredixcan-mashr-zscores-Breast_Mammary_Tissue-projection.pkl
  shape: (987, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


,score
count,411840.000000
mean,0.003638
std,0.024343
min,-0.946998
25%,-0.005213
50%,0.002992
75%,0.011164
max,1.024814


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-mashr-zscores-Breast_Mammary_Tissue-projection-prediction_scores.h5

spredixcan-mashr-zscores-Liver-projection.pkl
  shape: (987, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


...

    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-mashr-zscores-Vagina-projection-prediction_scores.h5

spredixcan-mashr-zscores-Whole_Blood-projection.pkl
  shape: (987, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


,score
count,411840.000000
mean,0.003653
std,0.025103
min,-0.794988
25%,-0.005360
50%,0.002933
75%,0.011320
max,1.090423


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-mashr-zscores-Whole_Blood-projection-prediction_scores.h5

spredixcan-most_signif-projection.pkl
  shape: (987, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


,score
count,411840.000000
mean,0.004380
std,0.028745
min,-1.266442
25%,-0.005748
50%,0.003558
75%,0.012919
max,1.113870


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-most_signif-projection-prediction_scores.h5

